In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import xlrd

# pollution

In [2]:
targetPattern = "./pollution/*.csv"
FileName_list = glob.glob(targetPattern)

In [43]:
SO2=[]
CO=[]
O3=[]
PM10=[]
PM25=[]
NO=[]
NOx=[]
year = []
month = []
city = []

In [44]:
for fna in FileName_list:
    try:
        df = pd.read_csv(fna)
        df_city = df['縣市'].value_counts().index
        for c in df_city:
            df_n = df[df['縣市']==c]
            SO2.append(None)
            SO2[-1] = np.mean(df_n[df_n['二氧化硫 SO2 (ppb)']!='-']['二氧化硫 SO2 (ppb)'].astype(float))
            CO.append(None)
            CO[-1] = np.mean(df_n[df_n['一氧化碳 CO (ppm)']!='-']['一氧化碳 CO (ppm)'].astype(float))
            O3.append(None)
            O3[-1] = np.mean(df_n[df_n['臭氧 O3 (ppb)']!='-']['臭氧 O3 (ppb)'].astype(float))
            PM10.append(None)
            PM10[-1] = np.mean(df_n[df_n.iloc[:,9]!='-'].iloc[:,9].astype(float))
            PM25.append(None)
            PM25[-1] = np.mean(df_n[df_n.iloc[:,10]!='-'].iloc[:,10].astype(float))
            NO.append(None)
            NO[-1] = np.mean(df_n[df_n['一氧化氮 NO (ppb)']!='-']['一氧化氮 NO (ppb)'].astype(float))
            NOx.append(None)
            NOx[-1] = np.mean(df_n[df_n['氮氧化物 NOx (ppb)']!='-']['氮氧化物 NOx (ppb)'].astype(float))
            year.append(None)
            year[-1] = fna[20:24]
            month.append(None)
            month[-1] = fna[25:27]
            city.append(None)
            city[-1] = c
    except:
        SO2_tmp = 0
        CO_tmp=0
        O3_tmp=0
        PM10_tmp=0
        PM25_tmp=0
        NO_tmp=0
        NOx_tmp=0
        s = False
        n=0
        f = open(fna,"r",encoding="utf-8")
        city_tmp = '臺北市'
        for i,line in enumerate(f):

            if line[0][0]=='1':
                s = True


            if s:
                try:
                    if line.split(',')[1] == city_tmp:
                        n+=1
                        SO2_tmp+=float(line.split(',')[4])
                        CO_tmp+=float(line.split(',')[5])
                        O3_tmp+=float(line.split(',')[7])
                        PM10_tmp+=float(line.split(',')[9])
                        PM25_tmp+=float(line.split(',')[10])
                        NO_tmp+=float(line.split(',')[12])
                        NOx_tmp+=float(line.split(',')[11])
                    else:
                        SO2.append(SO2_tmp/n)
                        CO.append(CO_tmp/n)
                        O3.append(O3_tmp/n)
                        PM10.append(PM10_tmp/n)
                        PM25.append(PM25_tmp/n)
                        NO.append(NO_tmp/n)
                        NOx.append(NOx_tmp/n)
                        city.append(city_tmp)
                        year.append(fna[20:24])
                        month.append(fna[25:27])
                        city_tmp = line.split(',')[1]

                        n=0

                except:
                    pass  

In [79]:
x = pd.DataFrame({'city':city, 'year':year, 'month':month, 'SO2':SO2[0:-2], 'CO':CO[0:-2], 'O3':O3[0:-2], 'PM10':PM10[0:-2], 'PM25':PM25[0:-1], 'NO':NO, 'NOx':NOx})

# Cancer

In [77]:
c_year = ['103', '104', '105', '106', '107']
file = f'./cancer/民國{c_year[0]}年縣市別主要癌症.xls'
wb = xlrd.open_workbook(filename=file)
C=[]
C33=[]
y_city = []
for c2 in df_city:
    sheet = wb.sheet_by_name(c2)
    C.append(sheet.cell(9,3).value)
    C33.append(sheet.cell(10,3).value)
    y_city.append(c2)

# Normalization

In [82]:
y_C = pd.DataFrame({'C':C, 'city':y_city})
y_C.C = y_C.C/max(y_C.C)
y_C33 = pd.DataFrame({'C33':C33, 'city':y_city})
y_C33.C33 = y_C33.C33/max(y_C33.C33)
x_ = x.drop(['city','year','month'],axis=1)
x_ = (x_ - x_.mean()) / (x_.max() - x_.min())

# DNN

In [92]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

In [96]:

# Placeholder
num_inputs = 7 # pollution
x_input = tf.placeholder(tf.float32, shape=[None, num_inputs])
y_input = tf.placeholder(tf.float32, shape=[None, 1])
learning_rate = 0.01

# layer
hidden_1 = fully_connected(x_input, 20, activation_fn=tf.nn.relu)
hidden_2 = fully_connected(hidden_1, 40, activation_fn=tf.nn.relu)
y_output = fully_connected(hidden_2, 1, activation_fn=tf.nn.relu)

# loss function
loss = tf.reduce_mean(tf.square(y_output - y_input))

# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train  = optimizer.minimize(loss)

# init
init = tf.global_variables_initializer()

In [ ]:
num_steps = 1000
with tf.Session() as sess:
    sess.run(init)
    for iteration in range(num_steps):
        sess.run(train,feed_dict={x_input: x_, y_input:y_})
        if iteration % 100 ==0:
            print()